# Importing transformer model for summarization tasks

Bart model from Hugging Face was chosen, as it is widely popular and well regarded.

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""
print(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[{'summary_text': 'Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002. She is believed to still be married to four men.'}]


In [ ]:
source = 'It is reported that on April, 23rd a lunar eclipse will be visible between 10PM and 11PM in the surroundings of Athens.'
summary = summarizer(source, max_length=130, min_length=30, do_sample=False)

Your max_length is set to 130, but your input_length is only 29. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=14)


In [ ]:
summary

[{'summary_text': 'On April, 23rd a lunar eclipse will be visible between 10PM and 11PM in the surroundings of Athens. It is reported that the eclipse will take place between the hours of 10pm and 11pm.'}]

# BLEU and ROUGE scores

For the consistency sake, for scores calculations library evaluate from hugging face was used.

https://huggingface.co/docs/evaluate/index

https://huggingface.co/spaces/evaluate-metric/bleu


In [ ]:
!pip install evaluate
!pip install rouge_score

In [ ]:
import evaluate

In [ ]:
bleu = evaluate.load("bleu")

In [93]:
# https://huggingface.co/spaces/evaluate-metric/bleu
def BLEU_score(source, summary):
    bleu_out = bleu.compute(predictions = summary, references = source)
    return round(bleu_out['bleu'],5), bleu_out

BLEU_score([source],[summary[0]['summary_text']])

(0.0,
 {'bleu': 1.4295017818212042e-06,
  'precisions': [0.9069767441860465,
   0.7380952380952381,
   0.6341463414634146,
   0.6],
  'brevity_penalty': 2.0122057092654536e-06,
  'length_ratio': 0.07084019769357495,
  'translation_length': 43,
  'reference_length': 607})

In [ ]:
rouge = evaluate.load('rouge')

In [ ]:
def ROUGE_score(source, summary, n=4):
    rouge_out = rouge.compute(predictions = summary, references = source)

    return round(rouge_out['rouge1'],5), rouge_out

ROUGE_score([source],[summary[0]['summary_text']])

(0.78571,
 {'rouge1': 0.7857142857142858,
  'rouge2': 0.7407407407407407,
  'rougeL': 0.6428571428571428,
  'rougeLsum': 0.6428571428571428})

In [85]:
BLEU_score(['A cat jumped onto the fence and it blinked'],['A cat blinked on the fence'])

(0.0,
 {'bleu': 0.0,
  'precisions': [0.8333333333333334, 0.4, 0.0, 0.0],
  'brevity_penalty': 0.6065306597126334,
  'length_ratio': 0.6666666666666666,
  'translation_length': 6,
  'reference_length': 9})

In [86]:
ROUGE_score(['A cat jumped onto the fence and it blinked'],['A cat blinked on the fence'])

(0.66667,
 {'rouge1': 0.6666666666666667,
  'rouge2': 0.3076923076923077,
  'rougeL': 0.5333333333333333,
  'rougeLsum': 0.5333333333333333})

In [95]:
ROUGE_score(['A cat jumped onto the fence and it blinked'],['cat blinked on the'])

(0.46154,
 {'rouge1': 0.46153846153846156,
  'rouge2': 0.0,
  'rougeL': 0.30769230769230765,
  'rougeLsum': 0.30769230769230765})

For this example, rouge1 is 0.46 and rouge2 0 - there are no 2-grams that are the same in both source and summary

In [84]:
BLEU_score(['A cat jumped onto the fence and it blinked'],['fence and it blinked'])

(0.2865,
 {'bleu': 0.2865047968601901,
  'precisions': [1.0, 1.0, 1.0, 1.0],
  'brevity_penalty': 0.2865047968601901,
  'length_ratio': 0.4444444444444444,
  'translation_length': 4,
  'reference_length': 9})

In [92]:
BLEU_score(['A cat jumped onto the fence and it blinked'],['and it blinked'])

(0.0,
 {'bleu': 0.0,
  'precisions': [1.0, 1.0, 1.0, 0.0],
  'brevity_penalty': 0.1353352832366127,
  'length_ratio': 0.3333333333333333,
  'translation_length': 3,
  'reference_length': 9})

In [87]:
ROUGE_score(['A cat jumped onto the fence and it blinked'],['fence and it blinked'])

(0.61538,
 {'rouge1': 0.6153846153846153,
  'rouge2': 0.5454545454545454,
  'rougeL': 0.6153846153846153,
  'rougeLsum': 0.6153846153846153})

# Preparation of examples

## Istruction on builidng a cupboard

In [59]:
source1 = """This project contains basic principles for building a frame for a basic cupboard. It can be expanded on by using different designs, the struts can be moved to accommodate shelves or simply left open plan. It will give you the idea of building storage space onto your home and can even be adapted to make a corner cupboard. The timber sizes can be changed and more struts can be put in to ensure greater strength if you feel you need it. The structure can be fixed to any wall, ceiling or floor using our guides to fixing. Check the wall, floor and ceiling area for pipes and cables before you drill anything. Tools you will need. Most of these can be found at the bottom of this page, the rest can be found by clicking on the toolstore link at the bottom of the page. We suggest for this simple cupboard you use 50 x 50mm PSE timber. This stands for Planed, Squared Edge and simply means prepared for joinery, as opposed to the "sawn" state timber is in when it leaves the timber mill. It is also called, in its prepared state, PAR which is Planed All Round. With the number of different door designs available from kitchen and DIY stores these days we suggest it may be a good idea for you to find the doors (if you intend to have them) first. The frame can easily be built around the door size and you will have guaranteed matching your existing decor. We have deliberately not given any dimensions in the project. Even when planed timber can vary by a mm or so and ceiling heights can vary considerably. Measure your own timber and other dimensions carefully and remember. Measure twice-Cut once. Use a carpenters square to make all cuts. Neat joints are impossible without a completely square face to begin with. Also if the timber is not square at the end, a measurement to the other end will vary, depending on which side you measure from. This can make it impossible to get a decent finish.
"""
summary1 = summarizer(source1, min_length=30, do_sample=False)
print(summary1)

[{'summary_text': 'This project contains basic principles for building a frame for a basic cupboard. The structure can be fixed to any wall, ceiling or floor using our guides to fixing. It can be expanded on by using different designs, the struts can be moved to accommodate shelves or simply left open plan.'}]


## Description of the Greenland shark



In [60]:
source2 = """The Greenland shark is the world’s longest living vertebrate. It can live for 400 years—twice the age of the longest-living land animal, the giant tortoise. There could be an individual in the ocean today that was alive during the 1665 Great Plague of London and George Washington’s presidential inauguration in 1789. Despite its name, the Greenland shark is not only found in Greenland; it has been spotted as far away as the western Caribbean. Also known as the gray, ground, gurry, or sleeper shark, this animal lives in incredibly cold, deep waters, so it is much less studied than many of the more than 500 other shark species. It is one of the slowest sharks, too: Its scientific name, Somniosus microcephalus, roughly translates to “sleepy small-head.” Yet these rare animals may be a top predator. Many shark species can be aged by counting growth bands on their vertebrae, like rings on a tree. The Greenland shark’s soft vertebrae do not have these bands, however. Instead, its age is determined by removing the layers of the lens of its eye—which continues growing throughout its lifetime—and radiocarbon dating the tissue in the center. Using this method, scientists discovered Greenland sharks have a life expectancy of at least 272 years and could reach 392, give or take about 120 years. What’s more, these slow-growing creatures don’t reach reproductive age until around 150 years old. Measuring the Greenland shark’s growth rate is challenging because individuals are rarely recaptured. However, one shark tagged in 1936 had only grown 2.3 inches when it reappeared 16 years later.
"""
summary2 = summarizer(source2, min_length=30, do_sample=False)
print(summary2)

[{'summary_text': 'The Greenland shark is the world’s longest living vertebrate. Also known as the gray, ground, gurry, or sleeper shark, this animal lives in incredibly cold, deep waters. It is one of the slowest sharks, too.'}]


## As bert was fine-tuned on CNN Daily Mail, lets use some examples test set of this dataset.

https://huggingface.co/datasets/cnn_dailymail

In [ ]:
!pip install datasets
from datasets import load_dataset
dataset = load_dataset(
   'cnn_dailymail', '3.0.0')

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
dataset['test'][0]

{'article': '(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC\'s founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians\' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday

In [61]:
source3 = dataset['test'][0]['article']
print(source3)
summary3 = summarizer(source3, min_length=30, do_sample=False)
print(summary3)

(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday's ceremony, sa

In [62]:
source4 = dataset['test'][5]['article']
print(source4)
summary4 = summarizer(source4, min_length=30, do_sample=False)
print(summary4)

(CNN)He's a blue chip college basketball recruit. She's a high school freshman with Down syndrome. At first glance Trey Moses and Ellie Meredith couldn't be more different. But all that changed Thursday when Trey asked Ellie to be his prom date. Trey -- a star on Eastern High School's basketball team in Louisville, Kentucky, who's headed to play college ball next year at Ball State -- was originally going to take his girlfriend to Eastern's prom. So why is he taking Ellie instead? "She's great... she listens and she's easy to talk to" he said. Trey made the prom-posal (yes, that's what they are calling invites to prom these days) in the gym during Ellie's P.E. class. Trina Helson, a teacher at Eastern, alerted the school's newspaper staff to the prom-posal and posted photos of Trey and Ellie on Twitter that have gone viral. She wasn't surpristed by Trey's actions. "That's the kind of person Trey is," she said. To help make sure she said yes, Trey entered the gym armed with flowers and 

## As a last example, lets use a book fragment from Hugging Face dataset airoboros-summarization

https://huggingface.co/datasets/mattpscott/airoboros-summarization

In [ ]:
dataset_books = load_dataset("mattpscott/airoboros-summarization")


Generating train split:   0%|          | 0/3949 [00:00<?, ? examples/s]

In [ ]:
dataset_books['train'][0]['instruction']

'BEGININPUT\nBEGINCONTEXT\nThere was no possibility of taking a walk that day. We had been wandering, indeed, in the leafless shrubbery an hour in the morning; but since dinner (Mrs. Reed, when there was no company, dined early) the cold winter wind had brought with it clouds so sombre, and a rain so penetrating, that further out-door exercise was now out of the question.\nI was glad of it: I never liked long walks, especially on chilly afternoons: dreadful to me was the coming home in the raw twilight, with nipped fingers and toes, and a heart saddened by the chidings of Bessie, the nurse, and humbled by the consciousness of my physical inferiority to Eliza, John, and Georgiana Reed.\nThe said Eliza, John, and Georgiana were now clustered round their mama in the drawing-room: she lay reclined on a sofa by the fireside, and with her darlings about her (for the time neither quarrelling nor crying) looked perfectly happy. Me, she had dispensed from joining the group; saying, "She regrett

In [63]:
source5 = dataset_books['train'][0]['instruction'][:3000]
summary5 = summarizer(source5, min_length=30, do_sample=False)
# print(summary5)

In [ ]:
source5

'BEGININPUT\nBEGINCONTEXT\nThere was no possibility of taking a walk that day. We had been wandering, indeed, in the leafless shrubbery an hour in the morning; but since dinner (Mrs. Reed, when there was no company, dined early) the cold winter wind had brought with it clouds so sombre, and a rain so penetrating, that further out-door exercise was now out of the question.\nI was glad of it: I never liked long walks, especially on chilly afternoons: dreadful to me was the coming home in the raw twilight, with nipped fingers and toes, and a heart saddened by the chidings of Bessie, the nurse, and humbled by the consciousness of my physical inferiority to Eliza, John, and Georgiana Reed.\nThe said Eliza, John, and Georgiana were now clustered round their mama in the drawing-room: she lay reclined on a sofa by the fireside, and with her darlings about her (for the time neither quarrelling nor crying) looked perfectly happy. Me, she had dispensed from joining the group; saying, "She regrett

In [ ]:
summary5

[{'summary_text': '"I was glad of it: I never liked long walks, especially on chilly afternoons," he says. "There is something truly forbidding in a child taking up her elders in that manner," he adds.'}]

In [64]:
examples = {'cupboard':(source1, summary1),
            'shark':(source2, summary2),
            'daily_ICC':(source3, summary3),
            'daily_basketball':(source4, summary4),
            'book':(source5, summary5)
}

In [79]:
scores = {}
for name, (source, summary) in examples.items():
    bleu_score = BLEU_score(['"'+source+'"'],['"'+summary[0]['summary_text']+'"'])
    rouge_score = ROUGE_score(['"'+source+'"'],['"'+summary[0]['summary_text']+'"'])
    scores[name] = {'bleu': bleu_score[1], 'rouge':rouge_score[1]}
print(scores)

{'cupboard': {'bleu': {'bleu': 0.003386047829227612, 'precisions': [1.0, 1.0, 0.9642857142857143, 0.9090909090909091], 'brevity_penalty': 0.0034993696951895244, 'length_ratio': 0.15025906735751296, 'translation_length': 58, 'reference_length': 386}, 'rouge': {'rouge1': 0.2543640897755611, 'rouge2': 0.2456140350877193, 'rougeL': 0.18952618453865339, 'rougeLsum': 0.18952618453865339}}, 'shark': {'bleu': {'bleu': 0.003832631046442978, 'precisions': [1.0, 0.9555555555555556, 0.8863636363636364, 0.8372093023255814], 'brevity_penalty': 0.0041765870149556685, 'length_ratio': 0.15436241610738255, 'translation_length': 46, 'reference_length': 298}, 'rouge': {'rouge1': 0.23225806451612904, 'rouge2': 0.21428571428571427, 'rougeL': 0.23225806451612904, 'rougeLsum': 0.23225806451612904}}, 'daily_ICC': {'bleu': {'bleu': 3.301219882662008e-07, 'precisions': [1.0, 0.8536585365853658, 0.675, 0.5641025641025641], 'brevity_penalty': 4.3720784247076354e-07, 'length_ratio': 0.0639269406392694, 'translation

## Scores summary

In [ ]:
import pandas as pd

In [91]:
results = []
for name, scores_out in scores.items():
    source_len = len(examples[name][0].split())
    summary_len = len(examples[name][1][0]['summary_text'].split())
    results.append([name, source_len, summary_len, round(scores_out['bleu']['bleu'],5), round(scores_out['rouge']['rouge2'],5)])

df = pd.DataFrame(results, columns=['example', 'source len', 'summary len', 'BLEU-4 score','ROUGE-2 score'])

In [89]:
df

,example,source len,summary len,BLEU score,ROUGE score
0,cupboard,349,51,0.00339,0.24561
1,shark,261,35,0.00383,0.21429
2,daily_ICC,567,37,0.00000,0.09462
3,daily_basketball,440,33,0.00002,0.10359
4,book,510,33,0.00000,0.09765


In [76]:
for name, scores_out in scores.items():
    print("="*40)
    print(f"Example: {name}")
    print("-"*20)
    print("BLEU results")
    print(scores_out['bleu'])
    print("-"*20)
    print(scores_out['rouge'])

Example: cupboard
--------------------
BLEU results
{'bleu': 0.002790780999301754, 'precisions': [1.0, 1.0, 0.9814814814814815, 0.9245283018867925], 'brevity_penalty': 0.002859401742022366, 'length_ratio': 0.14583333333333334, 'translation_length': 56, 'reference_length': 384}
--------------------
{'rouge1': 0.2543640897755611, 'rouge2': 0.2456140350877193, 'rougeL': 0.18952618453865339, 'rougeLsum': 0.18952618453865339}
Example: shark
--------------------
BLEU results
{'bleu': 0.003016202752509006, 'precisions': [1.0, 0.9534883720930233, 0.9047619047619048, 0.8536585365853658], 'brevity_penalty': 0.0032559449856858175, 'length_ratio': 0.14864864864864866, 'translation_length': 44, 'reference_length': 296}
--------------------
{'rouge1': 0.23225806451612904, 'rouge2': 0.21428571428571427, 'rougeL': 0.23225806451612904, 'rougeLsum': 0.23225806451612904}
Example: daily_ICC
--------------------
BLEU results
{'bleu': 1.6256795964265932e-07, 'precisions': [1.0, 0.8461538461538461, 0.7105263

## Summary

The results are a bit unexpected, as the best scores were not achieved for Daily Mail dataset even though it was the dataset used for this bart's fine-tuning, but BLEU scores are not comparable across different datasets (https://huggingface.co/spaces/evaluate-metric/bleu).
It could be attributed to the complexity or the length of the source, but cupboard example is not that shorter than the daily mail artcile about the basketball player. The text about shark and cupboard instruction are simpler and the same words appear more times. In contrast, daily mail in itself tries to be concise, so the content is less redundant.

BLEU scores are really small, for examples from hugging face datasets achieving basically 0. ROUGE scores look more sensible, but are still low. It can result from the model using n-grams that are not in a reference, which lowers the BLEU score (in contrast to ROUGE, which is recall based), but in our case, for BLEU precisions were high, but the score itself was really low - as the short summaries can have artificially high BLEU score, short summaries are penalised and that is the case for our examples.

In [77]:
source1

'This project contains basic principles for building a frame for a basic cupboard. It can be expanded on by using different designs, the struts can be moved to accommodate shelves or simply left open plan. It will give you the idea of building storage space onto your home and can even be adapted to make a corner cupboard. The timber sizes can be changed and more struts can be put in to ensure greater strength if you feel you need it. The structure can be fixed to any wall, ceiling or floor using our guides to fixing. Check the wall, floor and ceiling area for pipes and cables before you drill anything. Tools you will need. Most of these can be found at the bottom of this page, the rest can be found by clicking on the toolstore link at the bottom of the page. We suggest for this simple cupboard you use 50 x 50mm PSE timber. This stands for Planed, Squared Edge and simply means prepared for joinery, as opposed to the "sawn" state timber is in when it leaves the timber mill. It is also ca

In [78]:
source3

'(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC\'s founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians\' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday\'s ceremony